<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/378_GCO_DataLoading_Utils.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""Data loading utilities for Governance & Compliance Orchestrator

Loads agent action logs, policy rules, bias signals, and drift signals from JSON files.
"""

import json
from pathlib import Path
from typing import List, Dict, Any, Optional
from config import GovernanceComplianceOrchestratorConfig


def load_agent_action_logs(
    config: GovernanceComplianceOrchestratorConfig,
    agent_name: Optional[str] = None
) -> List[Dict[str, Any]]:
    """
    Load agent action logs from all batch files.

    Args:
        config: Configuration object with data file paths
        agent_name: Optional filter by agent name

    Returns:
        List of agent action log events
    """
    all_events = []
    data_dir = Path(config.data_dir)

    for log_file in config.agent_logs_files:
        file_path = data_dir / log_file
        if not file_path.exists():
            continue

        with open(file_path, 'r') as f:
            events = json.load(f)
            all_events.extend(events)

    # Filter by agent_name if specified
    if agent_name:
        all_events = [e for e in all_events if e.get("agent_name") == agent_name]

    return all_events


def load_policy_rules(config: GovernanceComplianceOrchestratorConfig) -> List[Dict[str, Any]]:
    """
    Load policy rules from JSON file.

    Args:
        config: Configuration object with data file paths

    Returns:
        List of policy rules
    """
    data_dir = Path(config.data_dir)
    file_path = data_dir / config.policy_rules_file

    if not file_path.exists():
        return []

    with open(file_path, 'r') as f:
        return json.load(f)


def load_bias_signals(config: GovernanceComplianceOrchestratorConfig) -> List[Dict[str, Any]]:
    """
    Load bias detection signals from JSON file.

    Args:
        config: Configuration object with data file paths

    Returns:
        List of bias signals
    """
    data_dir = Path(config.data_dir)
    file_path = data_dir / config.bias_signals_file

    if not file_path.exists():
        return []

    with open(file_path, 'r') as f:
        data = json.load(f)
        # Extract bias_signals array from the JSON structure
        return data.get("bias_signals", [])


def load_drift_signals(config: GovernanceComplianceOrchestratorConfig) -> List[Dict[str, Any]]:
    """
    Load drift and degradation signals from JSON file.

    Args:
        config: Configuration object with data file paths

    Returns:
        List of drift signals
    """
    data_dir = Path(config.data_dir)
    file_path = data_dir / config.drift_signals_file

    if not file_path.exists():
        return []

    with open(file_path, 'r') as f:
        data = json.load(f)
        # Extract drift_signals array from the JSON structure
        return data.get("drift_signals", [])


def build_policy_lookup(policies: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    Build fast lookup dictionary for policies by policy_id.

    Args:
        policies: List of policy rules

    Returns:
        Dictionary mapping policy_id to policy dict
    """
    return {p["policy_id"]: p for p in policies}


def build_events_lookup(events: List[Dict[str, Any]]) -> Dict[str, Dict[str, Any]]:
    """
    Build fast lookup dictionary for events by event_id.

    Args:
        events: List of agent action log events

    Returns:
        Dictionary mapping event_id to event dict
    """
    return {e["event_id"]: e for e in events}

